In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
passengerId = test['PassengerId']
survived = train['Survived']
train.drop(columns='Survived', inplace=True)
data = pd.concat([train, test]);
data.drop(columns='PassengerId', inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
Pclass      1309 non-null int64
Name        1309 non-null object
Sex         1309 non-null object
Age         1046 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1308 non-null float64
Cabin       295 non-null object
Embarked    1307 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 112.5+ KB


In [7]:
# Extracting titles from full names
data['Title'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [8]:
print('Unique titles: ', data['Title'].unique())

Unique titles:  ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


In [9]:
# Grouping the titles in more general groups
grouped_titles = {
    'Mr' :           'Mr',
    
    'Mrs':           'Mrs',
    'Mme':           'Mrs',
    'Ms':            'Mrs',
    
    'Miss':          'Miss',
    'Mlle':          'Miss',
    
    'Master':        'Master',
    
    'Don':           'Royalty',
    'Lady':          'Royalty',
    'Sir':           'Royalty',
    'the Countess':  'Royalty',
    'Jonkheer':      'Royalty',
    'Dona':          'Royalty',
    
    'Rev':           'Crew',
    'Dr':            'Crew',
    'Major':         'Crew',
    'Col':           'Crew',
    'Capt':          'Crew' 
}

In [10]:
# Updating titles
data['Title'] = data['Title'].map(grouped_titles)
data['Title'].value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Crew        23
Royalty      6
Name: Title, dtype: int64

In [11]:
# Imputing age
# We assume that people's age depends on their Sex, Pclass and Title
grouped = data.groupby(['Sex','Pclass', 'Title'])
print(grouped.Age.median())
# Transforming each group to dictionary key where values are median ages in that groups
dict_ages_grouped = grouped.Age.median().apply(lambda x: x).to_dict()

Sex     Pclass  Title  
female  1       Crew       49.0
                Miss       30.0
                Mrs        45.0
                Royalty    39.0
        2       Miss       20.0
                Mrs        30.0
        3       Miss       18.0
                Mrs        31.0
male    1       Crew       52.0
                Master      6.0
                Mr         41.5
                Royalty    40.0
        2       Crew       41.5
                Master      2.0
                Mr         30.0
        3       Master      6.0
                Mr         26.0
Name: Age, dtype: float64


In [12]:
dict_ages_grouped

{('female', 1, 'Crew'): 49.0,
 ('female', 1, 'Miss'): 30.0,
 ('female', 1, 'Mrs'): 45.0,
 ('female', 1, 'Royalty'): 39.0,
 ('female', 2, 'Miss'): 20.0,
 ('female', 2, 'Mrs'): 30.0,
 ('female', 3, 'Miss'): 18.0,
 ('female', 3, 'Mrs'): 31.0,
 ('male', 1, 'Crew'): 52.0,
 ('male', 1, 'Master'): 6.0,
 ('male', 1, 'Mr'): 41.5,
 ('male', 1, 'Royalty'): 40.0,
 ('male', 2, 'Crew'): 41.5,
 ('male', 2, 'Master'): 2.0,
 ('male', 2, 'Mr'): 30.0,
 ('male', 3, 'Master'): 6.0,
 ('male', 3, 'Mr'): 26.0}

In [13]:
for groups in dict_ages_grouped:
    data.loc[(data['Sex']==groups[0]) & 
           (data['Pclass']==groups[1]) &
           (data['Title']==groups[2]), 'Age'] = dict_ages_grouped[groups]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
Pclass      1309 non-null int64
Name        1309 non-null object
Sex         1309 non-null object
Age         1309 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1308 non-null float64
Cabin       295 non-null object
Embarked    1307 non-null object
Title       1309 non-null object
dtypes: float64(2), int64(3), object(6)
memory usage: 122.7+ KB
